In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse
from textblob import TextBlob
from nltk.corpus import cmudict
import syllables
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize

In [2]:
# Assuming your Excel file is named 'Input.xlsx'
input_file = 'Input.xlsx'

In [3]:
df = pd.read_excel(input_file)

In [4]:
#pip install beautifulsoup4

1. Data Extraction

In [5]:
# Function to check if the content resembles a URL
def is_url(s):
    try:
        result = urlparse(s)
        return all([result.scheme, result.netloc])
    except ValueError:
        return False

# Function to extract article text from HTML
def extract_article_text(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Assuming the article text is within the <body> tag
    article_text = soup.body.get_text(separator='\n', strip=True)
    
    return article_text

# Loop through each row in the DataFrame and fetch HTML content from URLs
for index, row in df.iterrows():
    url_id = row['URL_ID']
    
    # Make sure to check if the actual column name exists in your DataFrame
    try:
        html_content_or_url = row['URL']
    except KeyError:
        print(f"HTML_Content column not found in row {index}. Please check your column names.")
        continue
    
    # Check if the content resembles a URL
    if is_url(html_content_or_url):
        try:
            # Fetch HTML content from the URL
            response = requests.get(html_content_or_url)
            response.raise_for_status()  # Check for request success
            html_content = response.text
        except requests.exceptions.RequestException as e:
            print(f"Error fetching content from URL in row {index}: {e}")
            continue
    else:
        # If it's not a URL, assume it's already HTML content
        html_content = html_content_or_url
    
    # Extract article text using the function
    article_text = extract_article_text(html_content)
    
    # Save the extracted article text to a text file
    output_file = f'{url_id}.txt'
    with open(output_file, 'w', encoding='utf-8') as file:
        file.write(article_text)

print("Extraction and saving completed.")

Error fetching content from URL in row 35: 404 Client Error: Not Found for url: https://insights.blackcoffer.com/how-neural-networks-can-be-applied-in-various-areas-in-the-future/
Error fetching content from URL in row 48: 404 Client Error: Not Found for url: https://insights.blackcoffer.com/covid-19-environmental-impact-for-the-future/
Extraction and saving completed.


In [ ]:
#pip install textblob

#  Function to perform textual analysis and compute variables

In [ ]:
import pandas as pd
from textblob import TextBlob
from nltk.tokenize import sent_tokenize, word_tokenize
import syllables
import nltk

# Download necessary NLTK resources
# nltk.download('punkt')

# Read the extracted article texts and other input variables
df = pd.read_excel('Input.xlsx')  # Update with your actual file path

# Function to perform textual analysis and compute variables
def analyze_text(text):
    blob = TextBlob(text)
    sentences = sent_tokenize(text)
    words = word_tokenize(text)
    syllable_count = sum(syllables.estimate(word) for word in words)
    personal_pronouns = sum(1 for word, pos in blob.tags if pos == 'PRP')
    
    return {
        'POSITIVE_SCORE': blob.sentiment.polarity,
        'NEGATIVE_SCORE': -blob.sentiment.polarity,  # Invert polarity for negative score
        'POLARITY_SCORE': blob.sentiment.polarity,
        'SUBJECTIVITY_SCORE': blob.sentiment.subjectivity,
        'AVG_SENTENCE_LENGTH': sum(len(sent) for sent in sentences) / len(sentences) if len(sentences) > 0 else 0,
        'PERCENTAGE_OF_COMPLEX_WORDS': (sum(1 for word in words if len(word) > 2) / len(words)) * 100,
        'FOG_INDEX': 0.4 * (blob.sentiment.subjectivity + 100 * (sum(1 for word in words if len(word) > 2) / len(words))),
        'AVG_NUMBER_OF_WORDS_PER_SENTENCE': len(words) / len(sentences) if len(sentences) > 0 else 0,
        'COMPLEX_WORD_COUNT': sum(1 for word in words if len(word) > 2),
        'WORD_COUNT': len(words),
        'SYLLABLE_PER_WORD': syllable_count / len(words) if len(words) > 0 else 0,
        'PERSONAL_PRONOUNS': personal_pronouns,
        'AVG_WORD_LENGTH': sum(len(word) for word in words) / len(words) if len(words) > 0 else 0,
    }

# Apply the analysis function to each article text in the DataFrame
results = df['URL_ID'].apply(analyze_text)

# Create a DataFrame with the computed variables
output_df = pd.DataFrame(results.tolist())

# Concatenate the original DataFrame with the computed variables
final_output_df = pd.concat([df, output_df], axis=1)

# Save the final output DataFrame to an Excel file
final_output_df.to_excel('Output Data Structure.xlsx', index=False)


# saving output data in output data structure

In [15]:
# Assuming 'results' is a list of dictionaries with computed variables for each URL_ID
# Replace 'results' with your actual list of dictionaries

# Create a DataFrame with computed variables
output_df = pd.DataFrame(results)

# Assuming 'df' is your original DataFrame with the input variables
# Replace 'df' with your actual DataFrame

# Merge the computed variables with the original DataFrame based on 'URL_ID'
final_output_df = pd.merge(df, output_df, on='URL_ID')

# Save the final output DataFrame to an Excel file
final_output_df.to_excel(' Output Data Structure.xlsx', index=False)



In [24]:
# pip install syllables

In [34]:
# nltk.download('cmudict')

[nltk_data] Downloading package cmudict to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


True